# 1.preprocess

In [1]:
import sys
sys.path = ["../../.."] + sys.path # 切换到项目目录下

import scanpy as sc
import scvelo as scv
import velovgi

from ray import tune, air
from ray.air import session

Global seed set to 0
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


1. 目标函数

In [2]:
from pytorch_lightning import loggers
from torch_geometric import seed_everything

# TODO: 跳整多种参数，简化调参Trail的名字
def train_velovgi(config):
    # 提取参数
    # 随机数种子，确保结果的可复现性
    random_seed = config.get("random_seed", 0)
    # 预处理的参数
    n_bnn_neighbors = config.get("n_bnn_neighbors", 15)
    n_knn_neighbors = config.get("n_knn_neighbors", 15)
    is_ot = config.get("is_ot", True)
    # 模型结构参数
    n_hidden = config.get("n_hidden", 256)
    n_latent = config.get("n_latent", 10)
    n_layers = config.get("n_layers", 1)
    # 训练参数
    num_neighbors = [config.get("num_neighbors", 8)]*n_layers
    max_epochs = config.get("max_epochs", 10) # TODO:这里是最关键的一个参数，小epochs测试之后再提交到服务器上用大epoch
    batch_size = config.get("batch_size", 64)
    max_kl_weight = config.get("max_kl_weight", 0.8)

    name = ""
    for k,v in config.items():
        name += "%s_%s,"%(k, v)
    name = name[:-1]

    seed_everything(random_seed)
    # TODO:数据读入，对于不同的数据集这里需要替换
    adata_filename = "/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/erythroid_lineage.h5ad" # 数据路径使用绝对路径
    adata = scv.read(adata_filename)
    batch_key = "stage" # 批次key
    cluster_key = "celltype" # 细胞类型key
    cluster_edges = [
    ("Blood progenitors 1", "Blood progenitors 2"), 
    ("Blood progenitors 2", "Erythroid1"), 
    ("Erythroid1", "Erythroid2"), 
    ("Erythroid2", "Erythroid3")
    ] # 指定对应数据集已知的细胞类型间的分化信息

    # TODO:预处理，这里batch_pair_list以后可能需要手动指定
    batch_list = list(adata.obs[batch_key].cat.categories)
    batch_pair_list = list(zip(batch_list[:-1], batch_list[1:]))
    subsample_adata = velovgi.pp.preprocess(adata,
                                            n_bnn_neighbors=n_bnn_neighbors,
                                            n_knn_neighbors=n_knn_neighbors,
                                            batch_key=batch_key,
                                            batch_pair_list=batch_pair_list,
                                            is_ot=is_ot)
    # TODO: 如果需要调整预处理之后的参数，就不需要重复做预处理了，读取预处理之后的结果即可
    # adata = velovgi.tl.read_adata("/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata")
    # subsample_adata = scv.read("/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/subsample_adata.h5ad") # 使用这个AnnData做训练
    # seed_everything(random_seed)

    # 模型训练
    logger = loggers.TensorBoardLogger(save_dir="./log", name=name)
    velovgi.tl.VELOVGI.setup_anndata(adata=subsample_adata, spliced_layer="Ms", unspliced_layer="Mu")
    velovgi_model = velovgi.tl.VELOVGI(subsample_adata,
                                       n_hidden=n_hidden,
                                       n_latent=n_latent,
                                       n_layers=n_layers)
    velovgi_model.train(num_neighbors=num_neighbors,
                        max_epochs=max_epochs,
                        batch_size=batch_size,
                        plan_kwargs={"max_kl_weight": max_kl_weight},
                        logger=logger)

    # 模型恢复
    velovgi.tl.add_velovi_outputs_to_adata(subsample_adata, velovgi_model) # 模型输出
    velovgi.pp.moment_recover(adata, subsample_adata) # 恢复

    # 速率计算
    scv.tl.velocity_graph(adata)
    scv.pl.velocity_embedding(adata, color=cluster_key, title=name, save="arrow.png")
    scv.pl.velocity_embedding_stream(adata, color=cluster_key, title=name, legend_loc="right", save="stream.png")

    # 伪时间计算
    scv.tl.velocity_pseudotime(adata)
    scv.pl.velocity_embedding_stream(adata, color="velocity_pseudotime", title=name, colorbar=False, save="pseudotime.png")


    # 保存结果
    subsample_adata.write("subsample_adata.h5ad")
    velovgi.tl.write_adata(adata, "adata")
    velovgi_model.save("model")

    # 计算指标评价
    adata_velo = velovgi.tl.pre_metric(adata)
    exp_metrics = velovgi.tl.summary_metric(adata_velo, cluster_edges, cluster_key)[-1] # 计算指标汇总后的结果

    session.report({"CBDir": exp_metrics["CBDir"], "ICVCoh": exp_metrics["ICVCoh"]})


2. 搜索空间，这里可以添加键值，实现更多层面的网格调参

In [3]:
search_space = {
    # 随机数种子
    # "random_seed" : tune.grid_search([0, 2023]),
    # 预处理的参数
    # "n_bnn_neighbors" : tune.grid_search([10,20]),
    # "n_knn_neighbors" : tune.grid_search([10,20]),
    "is_ot" : tune.grid_search([True, False]),
    # 模型结构参数
    # "n_hidden" : tune.grid_search([128, 256]),
    # "n_latent" : tune.grid_search([8, 16]),
    # "n_layers" : tune.grid_search([1, 2]),
    # 训练参数
    # "num_neighbors" : tune.grid_search([2, 4]),
    "max_epochs" : tune.grid_search([10, 20, 30]),
    # "batch_size" : tune.grid_search([64, 128]),
    # "max_kl_weight" : tune.grid_search([0.6, 0.8]),
}

3. 执行调参，等待传入实验名称和搜索空间

In [4]:
from ray.tune.schedulers import ASHAScheduler

name = "grid_search" # TODO:指定此次调参的名字，这里是预处理过程的调参

tuner = tune.Tuner(
    train_velovgi,
    tune_config=tune.TuneConfig(
        metric="CBDir",
        mode="max",
        scheduler=ASHAScheduler()
    ),
    run_config=air.RunConfig(
        local_dir="./results", # Trail内部具体输出结果在这里保存
        name=name # 开启调参的Tensorboard日志
    ),
    param_space=search_space,
)

results = tuner.fit()

2023-06-10 11:14:16,901	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-10 11:14:18,648	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-06-10 11:14:18,661	INFO tensorboardx.py:172 -- pip install "ray[tune]" to see TensorBoard files.
2023-06-10 11:14:18,662	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(pid=6679) Global seed set to 0
(pid=6679) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=6679)   new_rank_zero_deprecation(
(pid=6679) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(pid=6679)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=6679) Filtered out 51490 genes that are detected 20 counts (shared).
(train_velovgi pid=6679) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=6679) Extracted 2000 highly variable genes.
(train_velovgi pid=6679) Logarithmized X.
(train_velovgi pid=6679) use_rep : X_pca


(train_velovgi pid=6679) WARNING: n_obs too small: adjusting to `n_neighbors = 1`
(pid=6748) Global seed set to 0
(pid=6748) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=6748)   new_rank_zero_deprecation(
(pid=6748) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(pid=6748)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=6748) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=6748) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=6748) Extracted 2000 highly variable genes.
(train_velovgi pid=6748) Logarithmized X.
(train_velovgi pid=6748) use_rep : X_pca


...

(train_velovgi pid=7010)   warn(
(train_velovgi pid=7010)   warn(


(train_velovgi pid=7010)     finished (0:00:02) --> added 
(train_velovgi pid=7010)     'velocity_graph', sparse matrix with cosine correlations (adata.uns)
(train_velovgi pid=7010) computing velocity embedding
(train_velovgi pid=7010)     finished (0:00:00) --> added
(train_velovgi pid=7010)     'velocity_umap', embedded velocity vectors (adata.obsm)
(train_velovgi pid=7010) saving figure to file ./figures/scvelo_arrow.png
(train_velovgi pid=7010) Figure(640x480)


(train_velovgi pid=7122) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
(train_velovgi pid=7122)   warn(


(train_velovgi pid=7122)   0%|          | 0/500 [00:00<?, ?cells/s]
(train_velovgi pid=7010) saving figure to file ./figures/scvelo_stream.png
(train_velovgi pid=7010) Figure(640x480)
(train_velovgi pid=7010) computing terminal states
(train_velovgi pid=7010)     identified 1 region of root cells and 1 region of end points .
(train_velovgi pid=7010)     finished (0:00:00) --> added
(train_velovgi pid=7010)     'root_cells', root cells of Markov diffusion process (adata.obs)
(train_velovgi pid=7010)     'end_points', end points of Markov diffusion process (adata.obs)
(train_velovgi pid=7122)     finished (0:00:00) --> added [repeated 2x across cluster]
(train_velovgi pid=7122) Figure(640x480)
(train_velovgi pid=7122) Figure(640x480)
(train_velovgi pid=7122) Figure(640x480)
(train_velovgi pid=7010) saving figure to file ./figures/scvelo_pseudotime.png
(train_velovgi pid=7122) Figure(640x480)
(train_velovgi pid=7010) Figure(640x480) [repeated 2x across cluster]
(train_velovgi pid=7010) cr

2023-06-10 11:20:55,192	INFO tune.py:945 -- Total run time: 396.54 seconds (396.46 seconds for the tuning loop).
